In [9]:
import pandas as pd
import re
import os
import csv


def LetterToNum(letter,mode='TP'):
    if mode == 'X':
        # -- X --    
        if letter == 'A+':
            note = 19
        elif letter == 'A':
            note = 17
        elif letter == 'A-':
            note = 15
        elif letter == 'B+':
            note = 14
        elif letter == 'B':
            note = 13
        elif letter == 'B-':
            note = 12
        elif letter == 'C':
            note = 11
        elif letter == 'D':
            note = 9
        elif letter == 'E':
            note = 7
        elif letter == 'F':
            note = 0
        else:
            note = None
    else:
        # -- Telecom --
        if letter == 'A+':
            note = 17.1
        elif letter == 'A':
            note = 16.9
        elif letter == 'A-':
            note = 13.9
        elif letter == 'B+':
            note = 13.9
        elif letter == 'B':
            note = 12.9
        elif letter == 'B-':
            note = 10.9
        elif letter == 'C':
            note = 9.9
        elif letter == 'D':
            note = 7.9
        elif letter == 'E':
            note = 5
        elif letter == 'F':
            note = 0
        else:
            note = None
    return note

MoodleFile = 'Copie de LAB103-2021 Grades_Moodle-1.xlsx'
SynapseSFile = 'Fichier Synapses LAB103 vierge.csv'

print("Saisir le nom du fichier de note Moodle :")
MoodleFile = input()
print("Saisir le nom du fichier Synapses :")
SynapseSFile = input()

dfMoodle = pd.read_excel(MoodleFile) 
dfSynapses = pd.read_csv(SynapseSFile,delimiter=';', encoding='utf-8-sig',skiprows=1, on_bad_lines='skip')

s1 = dfSynapses['Matricule']
s2 = dfMoodle['Institution']
intersection = pd.Series(list(set(s1) & set(s2)))

print("Les éléments suivants n'étaient pas dans Moodle")
print(dfMoodle[['Institution','First name','Surname','Email address']][~dfMoodle['Institution'].isin(intersection)])

dfMoodle['Course letter grade (Real)'].loc[dfMoodle['Institution'] == 'B210305']

for e in intersection:
    #print(dfMoodle.loc[dfMoodle['Institution'] == e]['Course letter grade (Real)'].values[0],'->',LetterToNum(dfMoodle.loc[dfMoodle['Institution'] == e]['Course letter grade (Real)'].values[0]))
    dfSynapses.at[dfSynapses['Matricule'] == e,'Note initiale'] = LetterToNum(dfMoodle.loc[dfMoodle['Institution'] == e]['Course letter grade (Real)'].values[0])
   
# on écrit le csv
dfSynapses.to_csv('out.csv',sep=';', encoding='utf-8', index=False)

with open("Fichier Synapses LAB103 vierge.csv",'r', encoding='utf-8-sig') as original:
    lines = []
    for line in original:
        lines.append(line)

with open("out.csv",'r') as csv:
    csvlines = []
    for line in csv:
        csvlines.append(line)

cours = lines[0].split(';')[0]        
        
with open (f'{cours}.csv', 'w', encoding='utf-8-sig') as fp:
    fp.write(lines[0])
    for l in csvlines:
        fp.write(l)
        
os.remove('out.csv')
print(f'\n\nLe fichier resultat est : {cours}.csv')

Saisir le nom du fichier de note Moodle :


 Copie de LAB103-2021 Grades_Moodle-1.xlsx


Saisir le nom du fichier Synapses :


 Fichier Synapses LAB103 vierge.csv


Les éléments suivants n'étaient pas dans Moodle
   Institution First name  Surname                   Email address
16         NaN       Jean  Boisson  jean.boisson@polytechnique.edu


Le fichier resultat est : LAB103.csv
